<a href="https://colab.research.google.com/github/EliasNoorzad/XAI_Autonomous-Driving/blob/main/evaluation/02_baseline_det_seg_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip -q install ultralytics==8.4.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00


In [3]:
!cp /content/drive/MyDrive/XAI_Project/BDD100K_640.zip /content/


In [4]:
!unzip -q /content/BDD100K_640.zip -d /content/BDD100K_640

In [5]:
yaml = """\
path: /content/BDD100K_640/yolo_640
train: train/images
val: val/images
test: test/images

nc: 5
names: [car, truck, bus, person, bike]
"""

with open("/content/BDD100K_640/yolo_640/dataset_640.yaml", "w") as f:
    f.write(yaml)


In [12]:
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset


class BDDDetDrivableDataset(Dataset):
    """
    For preprocessed 640 dataset (yolo_640 + drivable_masks_640):
      images: <yolo_root>/<split>/images/<stem>.jpg
      labels: <yolo_root>/<split>/labels/<stem>.txt
      masks : <mask_root>/<split>/<stem>.png

    Returns:
      img   : FloatTensor [3, H, W] in [0,1]
      labels: FloatTensor [N, 5] where each row is [cls, x, y, w, h] (YOLO normalized)
      mask  : FloatTensor [1, H, W] with values 0/1
    """
    def __init__(self, yolo_root: str, mask_root: str, split: str, imgsz: int = 640):
        self.yolo_root = Path(yolo_root)
        self.mask_root = Path(mask_root)
        self.split = split
        self.imgsz = int(imgsz)

        self.img_dir = self.yolo_root / split / "images"
        self.lbl_dir = self.yolo_root / split / "labels"
        self.msk_dir = self.mask_root / split

        if not self.img_dir.is_dir():
            raise FileNotFoundError(f"Missing images dir: {self.img_dir}")
        if not self.lbl_dir.is_dir():
            raise FileNotFoundError(f"Missing labels dir: {self.lbl_dir}")
        if not self.msk_dir.is_dir():
            raise FileNotFoundError(f"Missing masks dir:  {self.msk_dir}")

        exts = {".jpg", ".jpeg", ".png"}
        self.img_paths = sorted([p for p in self.img_dir.iterdir() if p.suffix.lower() in exts])
        if len(self.img_paths) == 0:
            raise FileNotFoundError(f"No images found in: {self.img_dir}")

    def __len__(self):
        return len(self.img_paths)

    @staticmethod
    def _read_yolo_labels(label_path: Path) -> torch.Tensor:
        # YOLO txt lines: class x y w h
        if not label_path.exists():
            return torch.zeros((0, 5), dtype=torch.float32)

        rows = []
        with open(label_path, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                if len(parts) != 5:
                    continue
                cls, x, y, w, h = parts
                rows.append([float(cls), float(x), float(y), float(w), float(h)])

        if len(rows) == 0:
            return torch.zeros((0, 5), dtype=torch.float32)
        return torch.tensor(rows, dtype=torch.float32)

    @staticmethod
    def _pil_to_chw_float(img: Image.Image) -> torch.Tensor:
        arr = np.array(img, dtype=np.float32) / 255.0  # HWC
        arr = np.transpose(arr, (2, 0, 1))  # CHW
        return torch.from_numpy(arr)

    def __getitem__(self, idx: int):
        img_path = self.img_paths[idx]
        stem = img_path.stem

        label_path = self.lbl_dir / f"{stem}.txt"
        mask_path = self.msk_dir / f"{stem}.png"

        if not mask_path.exists():
            raise FileNotFoundError(f"Missing mask for {stem}: {mask_path}")

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        # labels are already corrected for 640x640
        labels = self._read_yolo_labels(label_path)

        # optional safety check (won't change anything if already 640)
        if img.size != (self.imgsz, self.imgsz):
            img = img.resize((self.imgsz, self.imgsz), resample=Image.BILINEAR)
        if mask.size != (self.imgsz, self.imgsz):
            mask = mask.resize((self.imgsz, self.imgsz), resample=Image.NEAREST)

        img_t = self._pil_to_chw_float(img)  # [3,H,W]
        mask_np = (np.array(mask, dtype=np.uint8) > 0).astype(np.float32)
        mask_t = torch.from_numpy(mask_np)[None, :, :]  # [1,H,W]

        return img_t, labels, mask_t


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# IMPORTANT: make sure this imports your LOCAL ultralytics repo, not the installed package.
# Example (do this BEFORE importing ultralytics):
# import sys
# sys.path.insert(0, "/content/your_project/ultralytics")  # path that contains the "ultralytics/" folder

from ultralytics import YOLO


class YOLOv8DetSemSeg(nn.Module):
    """
    YOLOv8 detection model + tiny semantic seg head.
    Captures NECK features by hooking the Detect head INPUT (multi-scale features).
    """
    def __init__(self, yolo_weights: str = "yolov8n.pt"):
        super().__init__()
        self.yolo = YOLO(yolo_weights).model  # nn.Module
        self._neck_feats = None

        self.sem_head = nn.Sequential(
            nn.LazyConv2d(64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 1, kernel_size=1)
        )

        self._register_detect_input_hook()

    def _register_detect_input_hook(self):
        if not hasattr(self.yolo, "model"):
            raise RuntimeError("Unexpected Ultralytics model: no .model")

        detect_module = self.yolo.model[-1]
        name = detect_module.__class__.__name__.lower()
        if "detect" not in name:
            raise RuntimeError(f"Last module is not Detect (got {detect_module.__class__.__name__}).")

        # remove previous hook if exists
        if hasattr(self, "_detect_hook_handle") and self._detect_hook_handle is not None:
            self._detect_hook_handle.remove()

        def pre_hook(module, inputs):
            self._neck_feats = inputs[0]

        self._detect_hook_handle = detect_module.register_forward_pre_hook(pre_hook)


    @staticmethod
    def _pick_high_res_from_detect_inputs(feats):
        # feats: list/tuple of [B,C,H,W]
        if not isinstance(feats, (list, tuple)) or len(feats) == 0:
            raise RuntimeError("Detect input features not captured.")
        return max(feats, key=lambda t: t.shape[-2] * t.shape[-1])  # highest H*W (usually P3)

    def forward(self, x):
      # TRAIN: x is a batch dict -> YOLO returns (det_loss, loss_items)
      if isinstance(x, dict):
          self._neck_feats = None
          imgs = x["img"]
          det_loss, det_items = self.yolo(x)

          feat = self._pick_high_res_from_detect_inputs(self._neck_feats)
          seg_logits = self.sem_head(feat)
          seg_logits = F.interpolate(seg_logits, size=imgs.shape[-2:], mode="bilinear", align_corners=False)
          return det_loss, det_items, seg_logits

      # INFER: x is an image tensor -> YOLO returns preds
      self._neck_feats = None
      det_preds = self.yolo(x)

      feat = self._pick_high_res_from_detect_inputs(self._neck_feats)
      seg_logits = self.sem_head(feat)
      seg_logits = F.interpolate(seg_logits, size=x.shape[-2:], mode="bilinear", align_corners=False)
      return det_preds, seg_logits


In [14]:
import torch

def collate_det_seg(batch):
    imgs, labels_list, masks = zip(*batch)
    imgs = torch.stack(imgs, 0)
    masks = torch.stack(masks, 0)

    bboxes_all, cls_all, batch_idx_all = [], [], []
    for i, lab in enumerate(labels_list):
        if lab.numel() == 0:
            continue
        cls = lab[:, 0:1].long()     # FIX
        bboxes = lab[:, 1:5].float() # keep float
        bboxes_all.append(bboxes)
        cls_all.append(cls)
        batch_idx_all.append(torch.full((lab.shape[0],), i, dtype=torch.long))

    if len(bboxes_all):
        bboxes = torch.cat(bboxes_all, 0)
        cls = torch.cat(cls_all, 0)
        batch_idx = torch.cat(batch_idx_all, 0)
    else:
        bboxes = torch.zeros((0, 4), dtype=torch.float32)
        cls = torch.zeros((0, 1), dtype=torch.long)   # FIX
        batch_idx = torch.zeros((0,), dtype=torch.long)

    return {"img": imgs, "bboxes": bboxes, "cls": cls, "batch_idx": batch_idx, "mask": masks}

In [20]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) create wrapper
model = YOLOv8DetSemSeg(
    yolo_weights="/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best_fixednames.pt"
).to(device)

# 2) load your det+seg checkpoint (works whether best.pt is a state_dict or a ckpt dict)
ckpt = torch.load("/content/drive/MyDrive/XAI_Project/experiments/det_seg_noatt_640/weights/best.pt", map_location="cpu")
state = ckpt["state_dict"] if isinstance(ckpt, dict) and "state_dict" in ckpt else (ckpt["model"] if isinstance(ckpt, dict) and "model" in ckpt else ckpt)
model.load_state_dict(state, strict=False)

model.eval()



YOLOv8DetSemSeg(
  (yolo): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track

In [15]:
import torch

@torch.no_grad()
def val_iou(model, loader, device, max_batches=None):
    model.eval()
    total_iou = 0.0
    total_imgs = 0

    for bi, batch in enumerate(loader):
        if (max_batches is not None) and (bi >= max_batches):
            break

        img = batch["img"].to(device)
        gt  = (batch["mask"].to(device) > 0.5).float()

        _, seg_logits = model(img)
        pred = (torch.sigmoid(seg_logits) > 0.5).float()

        inter = (pred * gt).sum(dim=(1, 2, 3))
        union = ((pred + gt) > 0).float().sum(dim=(1, 2, 3)).clamp_min(1.0)

        iou = (inter / union)  # [B]
        total_iou += iou.sum().item()
        total_imgs += iou.numel()

    return total_iou / max(1, total_imgs)


In [17]:
test_ds = BDDDetDrivableDataset(
    yolo_root="/content/BDD100K_640/yolo_640",
    mask_root="/content/BDD100K_640/drivable_masks_640",
    split="test",
    imgsz=640
)

from torch.utils.data import DataLoader
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=8,
    shuffle=False,
    num_workers=2,
    pin_memory=False,
    collate_fn=collate_det_seg
)


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) build the wrapper (any YOLO weights just to initialize)
model = YOLOv8DetSemSeg(
    yolo_weights="/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best_fixednames.pt"
).to(device)

# 2) load det+seg checkpoint (overwrites everything)
sd = torch.load("/content/drive/MyDrive/XAI_Project/experiments/det_seg_noatt_640/weights/best.pt", map_location="cpu")
model.load_state_dict(sd, strict=True)
model.eval()


In [9]:
from ultralytics import YOLO
BASE = YOLO("/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best_fixednames.pt")
m_base = BASE.val(data="/content/BDD100K_640/yolo_640/dataset_640.yaml", imgsz=640, device=0, split="test")
print("BASE mAP50 =", m_base.box.map50, "mAP50-95 =", m_base.box.map)

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 32.6±17.5 MB/s, size: 69.5 KB)
val: Scanning /content/BDD100K_640/yolo_640/test/labels... 20000 images, 143 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 20000/20000 627.6it/s 31.9s
val: /content/BDD100K_640/yolo_640/test/images/e6f10c58-c46de527.jpg: 1 duplicate labels removed
val: New cache created: /content/BDD100K_640/yolo_640/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1250/1250 6.6it/s 3:09
                   all      20000     243649      0.646      0.478      0.531      0.328
                   car      19776     205093      0.747      0.687      0.744      0.458
                 truck       5500       8701      0.657       0.49      0.561      0.403
                   bus       2459       3217

In [10]:
import torch
from ultralytics import YOLO

sd = torch.load("/content/drive/MyDrive/XAI_Project/experiments/det_seg_noatt_640/weights/best.pt", map_location="cpu")

y = YOLO("/content/drive/MyDrive/XAI_Project/experiments/det_baseline/weights/best_fixednames.pt")
yolo_sd = {k.replace("yolo.", ""): v for k, v in sd.items() if k.startswith("yolo.")}
y.model.load_state_dict(yolo_sd, strict=True)

m_new = y.val(data="/content/BDD100K_640/yolo_640/dataset_640.yaml", imgsz=640, device=0, split="test")
print("NEW  mAP50 =", m_new.box.map50, "mAP50-95 =", m_new.box.map)

Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1339.0±503.2 MB/s, size: 66.5 KB)
val: Scanning /content/BDD100K_640/yolo_640/test/labels.cache... 20000 images, 143 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 20000/20000 5.6Git/s 0.0s
val: /content/BDD100K_640/yolo_640/test/images/e6f10c58-c46de527.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1250/1250 6.7it/s 3:07
                   all      20000     243649      0.625      0.467      0.507      0.303
                   car      19776     205093       0.78       0.66      0.739      0.444
                 truck       5500       8701      0.586      0.496      0.522      0.362
                   bus       2459       3217       0.62      0.423      0.465      0.335
                pers

In [21]:
import torch

best_pt = "/content/drive/MyDrive/XAI_Project/experiments/det_seg_noatt_640/weights/best.pt"
sd = torch.load(best_pt, map_location="cpu")

model.load_state_dict(sd, strict=True)
model.to(device).eval()

iou_full = val_iou(model, test_loader, device, max_batches=None)
print("test_iou_full =", iou_full)



test_iou_full = 0.6641148620903492
